In [ ]:
!pip3 install torch==1.10.0+cpu torchvision==0.11.1+cpu torchaudio==0.10.0+cpu -f https://download.pytorch.org/whl/cpu/torch_stable.html

In [ ]:
from pathlib import Path

import pandas as pd
import numpy as np

import torch
import torch.nn.functional as F

# SESSION-BASED RECOMMENDATIONS WITH RECURRENT NEURAL NETWORKS

https://arxiv.org/pdf/1511.06939v4.pdf

Abstract  
: 우리는 새로운 영역, 즉 추천 시스템에 순환 신경망(RNN)을 적용합니다. 실제 추천 시스템은 긴 사용자 이력(예: Netflix 시청기록) 대신 짧은 세션 기반 데이터(예: 작은 스포츠웨어 웹사이트 세션 기록)에 추천해야 하는 경우가 많습니다. 이 경우, 행렬 분해 접근 방식은 정확하지 않습니다. 이러한 문제는 일반적으로 항목간 추천(item-to-item), 즉 유사한 항목을 추천함으로써 해결됩니다. 우리는 전체 세션을 모델링함으로써 더 정확한 추천을 제공할 수 있다고 생각합니다. 따라서 세션 기반 권장 사항에 대한 RNN 기반 접근 방식을 제안합니다. 우리는 또한 task에 효율적인(practical) 방법을 도입하며, 위 문제를 더 유용하게 해결할 수 있는 ranking loss 함수와 같은 개선사항을 클래식 RNN에 적용합니다.
 두 데이터 세트에 대한 실험 결과는 기존에 사용되던 접근 방식에 비해 현저한 개선을 보여줍니다.

1. Introduction  
: 생략
2. Related work  
: 생략
3. RECOMMENDATIONS WITH RNNS  
: 순환 신경망은 가변 길이 시퀀스 데이터를 모델링하기 위해 고안되었습니다. RNN과 기존 Feed-Forward 모델의 주요 차이점은 네트워크를 구성하는 단위에 hidden state 상태가 존재한다는 것입니다. 기본 RNN은 다음 업데이트 기능을 사용하여 숨겨진 상태 h를 업데이트합니다.
$$
h_t = g(Wx_t + Uh_{t-1})
$$
여기서 g는 로지스틱 시그모이드 함수와 같은 smooth and bounded 함수입니다. $x_t$s는 단위 시간 t에서의 입력입니다.  RNN은 현재 상태 $h_t$가 주어지면 시퀀스의 다음 요소에 대한 확률 분포를 출력합니다.  
GRU(Cho et al., 2014)는 기울기 소실 문제를 해결하는 것을 목표로 하는 기본 RNN보다 정교한 모델입니다. GRU 게이트는 기본적으로 유닛의 hidden state를 업데이트할 시기와 양을 학습합니다. GRU의 활성화는 이전 활성화와 후보 활성화 $\hat{h_t}$ 사이의 선형 보간입니다.
$$
h_t = (1-z_t)h_{t-1}+z_t\hat{h_t}
$$
where the update gate is given by:
$$
z_t = \sigma{(W_zx_t + U_zh_{t-1})}
$$
while the candidate activation function $\hat{h_t}$ is computed in a similar manner:
$$
\hat{h_t} = tanh(Wx_t + U(r_t \odot h_{t-1})
$$
and finaly the reset gate $r_t$ is given by:
$$
r_t = \sigma(W_rx_t + U_rh_{t-1})
$$

3.1 Customizing the GRU model
<img src=https://i.ibb.co/RPVM70D/2021-11-04-01-21-30.png width=600></img>
: 세션 기반 추천 모델에 GRU 기반 RNN을 사용했습니다. 네트워크의 입력은 세션의 actual state이고 출력은 세션의 다음 이벤트 항목입니다. 세션의 상태는 실제 이벤트의 item이거나 지금까지 세션의 이벤트일 수 있습니다. 전자의 경우 1-of-N 인코딩이 사용됩니다. (즉, 입력 벡터의 길이는 item 수와 같고 활성 항목에 해당하는 좌표만 1이고 나머지는 0입니다.)  
후자는 더 일찍 발생한 이벤트에 discount되는 representations의 가중 합계를 사용합니다. 안정성을 위해 입력 벡터가 정규화됩니다. 우리는 이것이 메모리 효과를 향상시킬 것으로 기대합니다. RNN의 더 긴 메모리에 의해 잘 포착되지 않는 매우 국소적인 순서 제약의 강화입니다. 우리는 또한 추가 임베딩 레이어를 추가하는 실험을 했지만 1-of-N 인코딩이 항상 더 나은 성능을 보였습니다.  
네트워크의 핵심은 GRU 계층이며 마지막 계층과 출력 사이에 추가 피드포워드 계층을 추가할 수 있습니다. 출력은 항목의 예측된 선호도, 즉 각 항목에 대한 세션의 다음 항목이 될 가능성입니다. 여러 GRU 레이어를 사용하는 경우 이전 레이어의 은닉 상태가 다음 레이어의 입력이 됩니다. 입력은 네트워크의 더 깊은 GRU 레이어에 선택적으로 연결할 수도 있습니다. 이것이 성능을 향상시킨다는 것을 발견했기 때문입니다. 이벤트의 시계열 내에서 단일 이벤트의 표현을 묘사하는 그림 1의 전체 아키텍처를 참조하십시오.  
추천 시스템은 순환 신경망의 주요 응용 분야가 아니므로 작업에 더 적합하도록 기본 네트워크를 수정했습니다.  우리의 솔루션이 실제 환경에 적용될 수 있도록 실용적인 점도 고려했습니다.
  
3.1.1 Session-Parallel Mini-Baches
<img src=https://i.ibb.co/1bwBcKB/2021-11-04-01-21-37.png width=400></img>
자연어 처리 작업을 위한 RNN은 일반적으로 순차 미니 배치를 사용합니다.  예를 들어 문장의 단어 위에 슬라이딩 창을 사용하고 이러한 창 조각을 서로 옆에 놓아 미니 배치를 형성하는 것이 일반적입니다.  이것은 우리의 작업에 맞지 않습니다. 왜냐하면 (1) 세션의 길이 분포가 문장의 경우보다 훨씬 더 많이 다를 수 있습니다. 일부 세션은 단 2개의 이벤트로 구성되는 반면 다른 세션은 수백 개가 넘는 이벤트로 구성될 수 있습니다. (2) 우리의 목표는 세션이 시간이 지남에 따라 어떻게 발전하는지 캡처하는 것이므로 조각으로 나누는 것은 의미가 없습니다.  따라서 세션 병렬 미니 배치를 사용합니다.  먼저 세션에 대한 순서를 만듭니다.  그런 다음 첫 번째 X 세션의 첫 번째 이벤트를 사용하여 첫 번째 미니 배치의 입력을 형성합니다(원하는 출력은 활성 세션의 두 번째 이벤트입니다).  두 번째 미니 배치는 두 번째 이벤트 등으로 구성됩니다.  세션 중 하나가 종료되면 사용 가능한 다음 세션이 그 자리에 놓입니다.  세션은 독립적인 것으로 간주되므로 이 전환이 발생할 때 적절한 숨김 상태를 재설정합니다.  자세한 내용은 그림 2를 참조하십시오.

  
3.1.2 Sampling on the output
추천 시스템은 항목 수가 많을 때 특히 유용합니다. 중간 규모의 인터넷 쇼핑몰의 경우에도 수만개의 종류가 있지만 더 큰 사이트는 수십만 개 또는 수백만 개의 항목이 있는 경우가 많습니다. 각 단계에서 각 항목에 대한 점수를 계산하면 알고리즘이 항목 수와 이벤트 수의 곱으로 확장됩니다. 이것은 너무 커 실제로 사용할 수 없습니다. 따라서 출력을 샘플링하고 항목의 작은 하위 집합에 대한 점수만 계산해야 합니다. 이것은 또한 일부 가중치만 업데이트됨을 의미합니다. 원하는 출력 외에도 일부 부정적인 examples에 대한 점수를 계산하고 원하는 출력이 높은 순위가 되도록 가중치를 수정해야 합니다.  
arbitrary(임의) missing 이벤트의 자연스러운 해석(natural interpretation)은 사용자가 해당 항목의 존재를 알지 못하므로 상호 작용이 없었다고 보는 것입니다. 그러나 사용자가 항목에 대해 알고 있었고 항목을 싫어했기 때문에 상호 작용하지 않기로 선택했을 가능성은 낮습니다. 아이템의 인기가 높을수록 사용자가 알고 있을 가능성이 높기 때문에 누락된 이벤트가 싫어함을 표시할 가능성이 높습니다. 따라서 인기도에 비례하여 항목을 샘플링해야 합니다. 각 훈련 예제에 대해 별도의 샘플을 생성하는 대신 미니 배치의 다른 훈련 예제의 항목을 부정적인 예제로 사용합니다. 이 접근 방식의 이점은 샘플링을 건너뛰어 계산 시간을 더욱 줄일 수 있다는 것입니다. 또한 코드를 덜 복잡하게 만들어 더 빠른 매트릭스 작업으로 구현하는 측면에서도 이점이 있습니다. 한편, 이 접근 방식은 인기도 기반 샘플링이기도 합니다. 항목이 미니 배치의 다른 교육 예제에 포함될 가능성은 인기도에 비례하기 때문입니다. 

3.1.3 Ranking Loss

- BPR(Bayesian Personalized Ranking):
$$
L_s = -\frac{1}{N_S} \cdot \sum^{N_S}_{j=1} log(\sigma(\hat{r}_{s,i} - \hat{r}_{s,j}))
$$

- TOP1: 
$$
L_s = \frac{1}{N_S} \cdot \sum^{N_S}_{j=1} \sigma(\hat{r}_{s,j} - \hat{r}_{s,i}) + \sigma(\hat{r}^2_{s,j})
$$
  
4. EXPERIMENTS  
: 
2가지 데이터셋을 이용했고, 첫번째 데이터셋은 RecSys Challenge 2015. 2번째 데이터셋은 OTT video service platform. 첫번째 데이터셋의 경우, 길이 1 짜리 세션은 train, test set 모두 삭제했고, train set에 있지만 test set에 없는 아이템의 경우도 삭제했습니다. 
<img src=https://i.ibb.co/x36sDzN/2021-11-04-01-22-00.png width=400></img>
<img src=https://i.ibb.co/JQg5TZM/2021-11-04-01-22-04.png width=400></img>

5. Conclusion & Future work  
:

https://github.com/hungthanhpham94/GRU4REC-pytorch

https://github.com/yhs968/pyGRU4REC

# Data 전처리

In [ ]:
df = pd.read_csv('events.csv')
df = df.sort_values(['visitorid', 'timestamp']).reset_index(drop=True)
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
df['date'] = df['timestamp'].dt.strftime('%Y%m%d').astype(int)
df = df.drop(['transactionid'], axis=1)
len(df)

In [ ]:
# 각각 5% 정도의 크기를 val, test set으로 선정
train = df[df['date'] < 20150816]
val = df[(df['date'] >= 20150816) & (df['date'] < 20150819)]
test = df[df['date'] >= 20150819]

In [ ]:
print(len(train), len(val), len(test))

In [ ]:
df.head()

In [ ]:
def remove_session(df, session_id, way='short', size=200):
    session_size = df.groupby(session_id).size()
    if way == 'short':
        df = df[np.in1d(df[session_id], session_size[session_size > 1].index)]
    else:
        df = df[np.in1d(df[session_id], session_size[session_size < size].index)]
    return df

In [ ]:
train = remove_session(train, 'visitorid')

In [ ]:
train.groupby('visitorid').size().hist()

In [ ]:
train.groupby('visitorid').size()[(train.groupby('visitorid').size() > 200) & (train.groupby('visitorid').size() < 500)].hist()

In [ ]:
train.groupby('visitorid').size()[(train.groupby('visitorid').size() > 100) & (train.groupby('visitorid').size() < 200)].hist()

In [ ]:
train.groupby('visitorid').size()[(train.groupby('visitorid').size() > 50) & (train.groupby('visitorid').size() < 100)].hist()

In [ ]:
train = remove_session(train, 'visitorid', 'long')
train

In [ ]:
# 논문에서 제시한 방법대로 train에 있지만 test에 없는 item은 삭제한다.
val = val[np.in1d(val['itemid'], train['itemid'])]
test = test[np.in1d(test['itemid'], train['itemid'])]

# 논문에서 제시한 방법대로 세션 길이가 1인 항목은 삭제한다.
val = remove_session(val, 'visitorid')
test = remove_session(test, 'visitorid')

In [ ]:
print('Train Set has', len(train), 'Events, ', train['visitorid'].nunique(), 'Sessions, and', train['itemid'].nunique(), 'Items\n')
print('Val Set has', len(val), 'Events, ', val['visitorid'].nunique(), 'Sessions, and', val['itemid'].nunique(), 'Items\n')
print('Test Set has', len(test), 'Events, ', test['visitorid'].nunique(), 'Sessions, and', test['itemid'].nunique(), 'Items\n')

# Dataset 정의

In [ ]:
class Dataset:
    def __init__(self, df, session_id='visitorid', item_id='itemid', timestamp='timestamp'):
        self.df = df
        self.session_id = session_id
        self.item_id = item_id
        self.timestamp = timestamp
        self.add_item_idx()
        self.df = self.df.sort_values([self.session_id, self.timestamp])
        self.offsets = self.get_offset()
        self.session_idx = np.arange(self.df[self.session_id].nunique())
        self.items = self.item_map[self.item_id].unique()
        
    def get_offset(self):
        # session id마다 길이를 구하고, 해당 값을 누적합.
        # ex. 1, 1, 2, 2, 2, 3, 4,
        # => 0, 2, 3, 1, 1, 
        # => 0, 2, 5, 6, 7
        offsets = np.zeros(self.df[self.session_id].nunique() + 1, dtype=np.int32)
        offsets[1:] = self.df.groupby(self.session_id).size().cumsum()
        return offsets
    
    def add_item_idx(self):
        unique_items = self.df[self.item_id].unique()
        self.item_map = pd.DataFrame({'itemid': unique_items, 'item_idx': range(len(unique_items))})
        self.df = pd.merge(self.df, self.item_map, on=self.item_id, how='inner')

In [ ]:
class DataLoader:
    def __init__(self, dataset, batch_size=10):
        self.dataset = dataset
        self.batch_size = batch_size
        self.dataset.df = dataset.df

    def __iter__(self):

        df = self.dataset.df
        offsets = self.dataset.offsets
        session_idx = self.dataset.session_idx

        iters = np.arange(self.batch_size)
        max_iter = iters.max()
        start, end = offsets[session_idx[iters]], offsets[session_idx[iters] + 1]
        mask = []
        finished = False

        while not finished:
            minlen = (end - start).min()

            for i in range(minlen - 1):
                idx_input = df['item_idx'].values[start]
                idx_target = df['item_idx'].values[start + i + 1]
                input_ = torch.LongTensor(idx_input)
                target = torch.LongTensor(idx_target)
                yield input_, target, mask

            start = start + (minlen - 1)
            mask = np.arange(len(iters))[(end - start) <= 1]
            for idx in mask:
                max_iter += 1
                if max_iter >= len(offsets) - 1:
                    finished = True
                    break
                iters[idx] = max_iter
                start[idx] = offsets[session_idx[max_iter]]
                end[idx] = offsets[session_idx[max_iter] + 1]

In [ ]:
train_dataset = Dataset(train)
train_loader = DataLoader(train_dataset)

In [ ]:
val_dataset = Dataset(val)
val_loader = DataLoader(val_dataset)

In [ ]:
iterator = iter(train_loader)

In [ ]:
inputs, labels, mask =  next(iterator)
print(f'Model Input Item Idx are : {inputs}')
print(f'Label Item Idx are : {"":5} {labels}')
print(f'Previous Masked Input Idx are {mask}')

# Loss 함수 정의

In [ ]:
import torch

## BPR Loss 예시

In [ ]:
logit = torch.tensor([
    [1., 2., 3.],
    [4., 5., 6.],
    [7., 8., 9.]]
)

In [ ]:
logit.diag()

In [ ]:
logit.diag().view(-1, 1)

In [ ]:
logit.diag().view(-1, 1).expand_as(logit)

In [ ]:
logit.diag().view(-1, 1).expand_as(logit) - logit

In [ ]:
F.logsigmoid(logit.diag().view(-1, 1).expand_as(logit) - logit)

In [ ]:
loss = -torch.mean(F.logsigmoid(logit.diag().view(-1, 1).expand_as(logit) - logit))
loss

## TOP 1 Loss 예시

In [ ]:
diff = -(logit.diag().view(-1, 1).expand_as(logit) - logit)

In [ ]:
F.sigmoid(diff).mean()

In [ ]:
F.sigmoid(diff ** 2).mean()

In [ ]:
loss = F.sigmoid(diff).mean() + F.sigmoid(diff ** 2).mean()
loss

In [ ]:
def BPRLoss(logit):
    diff = logit.diag().view(-1, 1).expand_as(logit) - logit
    loss = -torch.mean(F.logsigmoid(diff))

    return loss
    

    
def TOP1Loss(logit):
    diff = -(logit.diag().view(-1, 1).expand_as(logit) - logit)
    loss = F.sigmoid(diff).mean() + F.sigmoid(logit ** 2).mean()

    return loss

# Metric 정의

In [ ]:
def get_recall(indices, targets):
    targets = targets.view(-1, 1).expand_as(indices)
    hits = (targets == indices).nonzero()
    if len(hits) == 0:
        return 0
    n_hits = (targets == indices).nonzero()[:, :-1].size(0)
    recall = float(n_hits) / targets.size(0)
    return recall


def get_mrr(indices, targets):
    tmp = targets.view(-1, 1)
    targets = tmp.expand_as(indices)
    hits = (targets == indices).nonzero()
    ranks = hits[:, -1] + 1
    ranks = ranks.float()
    rranks = torch.reciprocal(ranks)
    mrr = torch.sum(rranks).data / targets.size(0)
    return mrr


def evaluate(indices, targets, k=20):
    _, indices = torch.topk(indices, k, -1)
    recall = get_recall(indices, targets)
    mrr = get_mrr(indices, targets)
    return recall, mrr

# 모델 정의

In [ ]:
from torch import nn
import torch

class GRU4REC(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1,
                 dropout_hidden=.5, dropout_input=0, batch_size=50, use_cuda=False):
        super(GRU4REC, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.num_layers = num_layers
        self.dropout_hidden = dropout_hidden
        self.dropout_input = dropout_input
        self.batch_size = batch_size
        self.use_cuda = use_cuda
        self.device = torch.device('cuda' if use_cuda else 'cpu')
        self.onehot_buffer = self.init_emb()
        self.h2o = nn.Linear(hidden_size, output_size)
        self.final_activation = nn.Tanh()
        self.gru = nn.GRU(self.input_size, self.hidden_size, self.num_layers, dropout=self.dropout_hidden)
        self = self.to(self.device)


    def forward(self, inp, hidden):

        embedded = self.onehot_encode(inp)
        if self.training and self.dropout_input > 0:
            embedded = self.embedding_dropout(embedded)
        embedded = embedded.unsqueeze(0)

        output, hidden = self.gru(embedded, hidden) #(num_layer, B, H)
        output = output.view(-1, output.size(-1))  #(B,H)
        logit = self.final_activation(self.h2o(output))

        return logit, hidden

    def init_emb(self):
        onehot_buffer = torch.FloatTensor(self.batch_size, self.output_size)
        onehot_buffer = onehot_buffer.to(self.device)
        return onehot_buffer

    def onehot_encode(self, inp):
        self.onehot_buffer.zero_()
        index = inp.view(-1, 1)
        one_hot = self.onehot_buffer.scatter_(1, index, 1)
        return one_hot

    def embedding_dropout(self, inp):
        p_drop = torch.Tensor(inp.size(0), 1).fill_(1 - self.dropout_input)
        mask = torch.bernoulli(p_drop).expand_as(inp) / (1 - self.dropout_input)
        mask = mask.to(self.device)
        inp = inp * mask
        return inp

    def init_hidden(self):
        return torch.zeros(self.num_layers, self.batch_size, self.hidden_size).to(self.device)

## optimizer

In [ ]:
import torch.optim as optim


class Optimizer:
    def __init__(self, params, lr=.05, momentum=0, weight_decay=0, eps=1e-6):
        self.optimizer = optim.Adagrad(params, lr=lr, weight_decay=weight_decay)
        
    def zero_grad(self):
        self.optimizer.zero_grad()

    def step(self):
        self.optimizer.step()

# 모델 훈련

In [ ]:
import numpy as np
import torch
from tqdm import tqdm

class Evaluation:
    def __init__(self, model, loss_func, use_cuda, k=20):
        self.model = model
        self.loss_func = loss_func
        self.topk = k
        self.device = torch.device('cuda' if use_cuda else 'cpu')

    def eval(self, eval_data, batch_size):
        self.model.eval()
        losses = []
        recalls = []
        mrrs = []
        dataloader = DataLoader(eval_data, batch_size)
        with torch.no_grad():
            hidden = self.model.init_hidden()
            for ii, (input, target, mask) in tqdm(enumerate(dataloader), total=len(dataloader.dataset.df) // dataloader.batch_size, miniters = 1000):
                input = input.to(self.device)
                target = target.to(self.device)
                logit, hidden = self.model(input, hidden)
                logit_sampled = logit[:, target.view(-1)]
                loss = self.loss_func(logit_sampled)
                recall, mrr = evaluate(logit, target, k=self.topk)

                losses.append(loss.item())
                recalls.append(recall)
                mrrs.append(mrr)
        mean_losses = np.mean(losses)
        mean_recall = np.mean(recalls)
        mean_mrr = np.mean(mrrs)

        return mean_losses, mean_recall, mean_mrr

In [ ]:
import os
import time
import torch
import numpy as np
from tqdm import tqdm


class Trainer:
    def __init__(self, model, train_data, eval_data, optim, use_cuda, loss_func, batch_size):
        self.model = model
        self.train_data = train_data
        self.eval_data = eval_data
        self.optim = optim
        self.loss_func = loss_func
        self.evaluation = Evaluation(self.model, self.loss_func, use_cuda, k=20)
        self.device = torch.device('cuda' if use_cuda else 'cpu')
        self.batch_size = batch_size

    def train(self, end_epoch, start_time=None):
        if start_time is None:
            self.start_time = time.time()
        else:
            self.start_time = start_time

        for epoch in range(1, end_epoch+1):
            st = time.time()
            print('Start Epoch #', epoch)
            train_loss = self.train_epoch(epoch)
            loss, recall, mrr = self.evaluation.eval(self.eval_data, self.batch_size)
            print("Epoch: {}, train loss: {:.4f}, loss: {:.4f}, recall: {:.4f}, mrr: {:.4f}, time: {}".format(epoch, train_loss, loss, recall, mrr, time.time() - st))
            checkpoint = {
                'model': self.model,
                'epoch': epoch,
                'optim': self.optim,
                'loss': loss,
                'recall': recall,
                'mrr': mrr
            }
            model_name = f"model_epoch{epoch}.pt"
            torch.save(checkpoint, model_name)
            print("Save model as %s" % model_name)


    def train_epoch(self, epoch):
        self.model.train()
        losses = []

        def reset_hidden(hidden, mask):
            if len(mask) != 0:
                hidden[:, mask, :] = 0
            return hidden

        hidden = self.model.init_hidden()
        dataloader = DataLoader(self.train_data, self.batch_size)
        for ii, (input, target, mask) in tqdm(enumerate(dataloader), total=len(dataloader.dataset.df) // dataloader.batch_size, miniters = 1000):
            input = input.to(self.device)
            target = target.to(self.device)
            self.optim.zero_grad()
            hidden = reset_hidden(hidden, mask).detach()
            logit, hidden = self.model(input, hidden)
            # output sampling
            logit_sampled = logit[:, target.view(-1)]
            loss = self.loss_func(logit_sampled)
            losses.append(loss.item())
            loss.backward()
            self.optim.step()

        mean_losses = np.mean(losses)
        return mean_losses

In [ ]:
input_size = len(train_dataset.items)
hidden_size = 100
output_size = input_size

In [ ]:
model = GRU4REC(input_size, hidden_size, output_size)

In [ ]:
optimizer = Optimizer(model.parameters())

In [ ]:
trainer = Trainer(model, train_dataset, val_dataset, optimizer, False, BPRLoss, 50)
trainer.train(1)

# 테스트

In [ ]:
checkpoint = torch.load('model_epoch1.pt')

In [ ]:
loaded_model = checkpoint["model"]
loaded_model.gru.flatten_parameters()
evaluation = Evaluation(loaded_model, BPRLoss, use_cuda=False, k=20)
loss, recall, mrr = evaluation.eval(val_dataset, 50)
print(f"Final result: recall = {round(recall, 2)}, mrr = {round(mrr, 2)}")